# Interpolação Paramétrica de Margens Foliares com Splines

Folhas são um dos mais importantes órgãos estruturais e funcionais das plantas, e as modelar computacionalmente é importante para o estudo de diversos problemas, como: Análise morfométrica das plantas; Reconstrução 3D da cobertura formada por copas de árvores; Estimar quantidade de luz que atravessa copas de árvores; Simular distribuição de água e luz nas folhas; Modelar resíduos de pesticidas na superficie foliar; Análise de crescimento e resposta das plantas; Entre outras.

Existem diversas técnicas para a reconstrução de folhas (e plantas como um todo), porém aplicadas à reconstrução 3D para usos como jogos digitais, por exemplo. Embora sejam representações realistas, deixam a desejar no quesito acurácia. Para aplicações que demandam simulações de ambiente, por exemplo, são necessários modelos extremamente precisos, extraídos de fontes reais.

## Escopo

Este documento se motiva do artigo "[A Leaf Modeling and Multi-Scale Remeshing Method for Visual Computation via Hierarchical Parametric Vein and Margin Representation](https://www.frontiersin.org/journals/plant-science/articles/10.3389/fpls.2018.00783/full)", em que é aplicado a técnica de reconstrução de superfícies de plantas para a modelagem estrutural funcional delas. A abordagem utilizada pode ser descrita com os seguintes passos:

- Aquisição de dados: Dada uma folha real, técnicas como Scan 3D com pré-processamento são utilizadas;
- Extração de características: Selecionar de forma hierárquica pontos importantes que definem o esqueleto da planta, formado por veias e margens, conectando-as com B-Splines;
- Modelagem paramétrica de superfície: Criar uma superfície auxiliar utilizando NURBS definida pelas curvas geradas anteriormente;
- Reamostragem e geração de malha: Pontos importantes são extraídos das curvas e o modelo de superfície, gerando uma malha a partir dessa extração;
- Otimização da malha: Gerar diferentes resoluções para a malha, com o objetivo de equilibrar eficiência computacional e preservação de detalhes.

![img](https://www.frontiersin.org/files/Articles/276320/fpls-09-00783-HTML/image_m/fpls-09-00783-g001.jpg)

Aqui daremos foco às duas primeiras etapas, de uma forma simplificada:

- Aquisição de dados: Dada a imagem de uma folha real, aplicaremos algoritmos de visão computacional para extrair suas margens;
- Extração de características: Obtidos pontos principais da margem, resolveremos o problema de interpolação com uma curva B-Spline periódica (fechada e contínua).

Mesmo simplificado, teremos uma base de estudo para a resolução do problema de modelagem de folhas.

## Sobre o problema de interpolação de pontos com uma B-spline periódica

O problema de se interpolar pontos com uma B-spline períodica se resume a encontrar os coeficiêntes dos polinômios que definem a curva, de modo que ela passe pelos pontos dados além de ser fechada e contínua. Podemos representar isso como um sistema linear, onde as incógnitas são os coeficientes de dontrole da Spline.

Por ser periódica, sua continúidade e fechamendo fazem com que o sistema linear resultante seja de banda cíclica, ou seja, um sistema linear cuja matriz é predominantemente composta por diagonais principais e secundárias (como uma matriz tridiagonal), mas com elementos não nulos conectando o início e o fim das diagonais, formando um ciclo.

 Isso reflete a necessidade de garantir que a curva spline seja suave e contínua ao fechar sobre si mesma, conectando o último ponto ao primeiro. Um exemplo de sistema linear de banda cíclica (para 5 incógnitas) pode ser representado por:

$$
\begin{bmatrix}
a & b & 0 & 0 & c \\
c & a & b & 0 & 0 \\
0 & c & a & b & 0 \\
0 & 0 & c & a & b \\
b & 0 & 0 & c & a \\
\end{bmatrix}
\begin{bmatrix}
x_1 \\
x_2 \\
x_3 \\
x_4 \\
x_5 \\
\end{bmatrix}
=
\begin{bmatrix}
b_1 \\
b_2 \\
b_3 \\
b_4 \\
b_5 \\
\end{bmatrix}
$$

Note que os elementos \( b \) e \( c \) conectam o início e o fim da matriz, formando o ciclo característico desse tipo de sistema.


# Aquisição de dados

# Interpolação de pontos

## B-Splines

Essas curvas são definidas como combinações lineares de funções polinomiais por partes, e possuem três elementos principais:

- Conjunto de pontos de controle
- Grau polinomial
- Vetor de nós

B-Splines não necessariamente passam pelos pontos de controle, apenas as utilizam para definir sua forma.

Seja o vetor de nós $T = \{t_0, t_1, \dots, t_m\}$, onde $T$ é uma sequência não decrescente com $t_i \in [0, 1]$. Agora defina os pontos de controls $P_0, P_1, \dots, P_n$. Defina o grau como $p = m - n - 1$.

Agora defina a função base como:
$$
N_{i,0}(t) =
\begin{cases}
1 \quad \text{se } t_i \leq t \leq t_{i+1} \\
0 \quad \text{caso contrário}
\end{cases}
$$

$$
N_{i,j}(t) = \frac{t-t_i}{t_{i+j} - t_i} N_{i, j-1}(t) + \frac{t_{i+j+1} - t}{t_{i+j+1} - t_{j-1}} N_{i+1, j-1}(t)
$$

Onde $j = 1, 2, \dots, p$. Então a curva definida por
$$
C(t) = \sum_{i=0}^n P_i N_{i,p}(t)
$$
é uma B-spline.

### Interpolação com B-Splines

O problema de interpolação com splines consiste em encontrar uma B-spline que passe exatamente por um conjunto de pontos $X$. Isso resulta em um sistema de equações, necessário para encontrar os coeficientes dos polinômios que compõe a curva.